In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [4]:
import numpy as np
import pandas as pd
import datetime as dt
import sqlite3

# Reflect Tables into SQLAlchemy ORM

In [5]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import desc

In [6]:
# create engine to hawaii.sqlite
engine = create_engine("path to sqlite file") # get this from git hub

In [6]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)


# reflect the tables
Recalls = Base.classes.recalls


NameError: name 'engine' is not defined

In [8]:
# View all of the classes that automap found
all_classes = Base.classes
for cls_name in all_classes.keys():
    print(cls_name)

measurement
station


In [9]:
# Save references to each table
Recalls = Base.classes.station


In [10]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Drug Recall Analysis

In [11]:
# Find the most recent date in the data set.
most_recent_date = session.query(Measurement.date).order_by(desc(Measurement.date)).first()

# Extract the date value from the result
most_recent_date = most_recent_date[0] if most_recent_date else None

print("The most recent date in the dataset is:", most_recent_date)


The most recent date in the dataset is: 2017-08-23


In [ ]:
annual_recalls = session.query(recalls.Recall_begins, recalls.event_id)

query = '''
        SELECT strftime('%Y', recall_date) as year, COUNT(*) as num_recalls
        FROM recalls
        GROUP BY year
        ORDER BY year
        '''

# Execute the query and load data into a DataFrame
df = pd.read_sql(query, conn)

# Close the database connection
conn.close()

# Convert year column to datetime
df['year'] = pd.to_datetime(df['year'])

# Plot the trend line
plt.figure(figsize=(10, 6))
plt.plot(df['year'], df['num_recalls'], marker='o')
plt.title('Recalls by Year')
plt.xlabel('Year')
plt.ylabel('Number of Recalls')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
#Query to show bar graph of # of recalls by reason

# Write your SQL query to retrieve the necessary data
query = '''
        SELECT Substance_Name, COUNT(*) as num_recalls
        FROM recalls
        GROUP BY Reason
        ORDER BY num_recalls DESC
        '''

# Execute the query and load data into a DataFrame
df = pd.read_sql(query, conn)

# Close the database connection
conn.close()

# Plot the bar chart
plt.figure(figsize=(10, 6))
plt.bar(df['Substance_Name'], df['num_recalls'])
plt.title('Volume of Recalls by Reason')
plt.xlabel('Reason')
plt.ylabel('Number of Recalls')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


In [ ]:
#code to show in bar graph the number of recalls by recalling_firm

# Write your SQL query to retrieve the necessary data
query = '''
        SELECT Recalling_Firm, COUNT(*) as num_recalls
        FROM recalls
        GROUP BY Recalling_Firm
        ORDER BY num_recalls DESC
        '''

# Execute the query and load data into a DataFrame
df = pd.read_sql(query, conn)

# Close the database connection
conn.close()

# Plot the bar chart
plt.figure(figsize=(10, 6))
plt.bar(df['Recalling_Firm'], df['num_recalls'])
plt.title('Number of Recalls by Recalling Firm')
plt.xlabel('Recalling Firm')
plt.ylabel('Number of Recalls')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


In [ ]:
#code to display the % of recalls by classification 

# Write your SQL query to retrieve the necessary data
query = '''
        SELECT Classification, COUNT(*) as num_recalls
        FROM recalls
        GROUP BY Classification
        ORDER BY num_recalls DESC
        '''

# Execute the query and load data into a DataFrame
df = pd.read_sql(query, conn)

# Calculate the percentage of recalls for each classification
df['percentage'] = (df['num_recalls'] / df['num_recalls'].sum()) * 100

# Close the database connection
conn.close()

# Plot the bar chart
plt.figure(figsize=(10, 6))
plt.bar(df['Classification'], df['percentage'])
plt.title('Percentage of Recalls by Classification')
plt.xlabel('Classification')
plt.ylabel('Percentage of Recalls')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


In [ ]:
#code to show % of recalls that were voluntary vs mandated

# Write your SQL query to retrieve the necessary data
query = '''
        SELECT Voluntary_Mandated, COUNT(*) as num_recalls
        FROM recalls
        GROUP BY Voluntary_Mandated
        ORDER BY num_recalls DESC
        '''

# Execute the query and load data into a DataFrame
df = pd.read_sql(query, conn)

# Calculate the percentage of recalls for each category
df['percentage'] = (df['num_recalls'] / df['num_recalls'].sum()) * 100

# Close the database connection
conn.close()

# Plot the bar chart
plt.figure(figsize=(6, 6))
plt.bar(df['Voluntary_Mandated'], df['percentage'])
plt.title('Percentage of Recalls by Voluntary_Mandated')
plt.xlabel('Voluntary/Mandated')
plt.ylabel('Percentage of Recalls')
plt.tight_layout()
plt.show()


In [ ]:
#code to show the average time (in years) that a recall lasted by reason type

# Write your SQL query to retrieve the necessary data
query = '''
        SELECT Reason, AVG((julianday(Recall_End) - julianday(Recall_Begins)) / 365) AS avg_duration_years
        FROM recalls
        WHERE Recall_End IS NOT NULL AND Recall_Begins IS NOT NULL
        GROUP BY Reason
        ORDER BY avg_duration_years DESC
        '''

# Execute the query and load data into a DataFrame
df = pd.read_sql(query, conn)

# Close the database connection
conn.close()

# Plot the bar chart
plt.figure(figsize=(10, 6))
plt.bar(df['Reason'], df['avg_duration_years'])
plt.title('Average Recall Duration by Reason')
plt.xlabel('Reason')
plt.ylabel('Average Duration (Years)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


In [ ]:
#code to show the % of recalls by state

# Write your SQL query to retrieve the necessary data
query = '''
        SELECT State, COUNT(*) as num_recalls
        FROM recalls
        GROUP BY State
        ORDER BY num_recalls DESC
        '''

# Execute the query and load data into a DataFrame
df = pd.read_sql(query, conn)

# Calculate the percentage of recalls for each state
df['percentage'] = (df['num_recalls'] / df['num_recalls'].sum()) * 100

# Close the database connection
conn.close()

# Plot the bar chart
plt.figure(figsize=(12, 6))
plt.bar(df['State'], df['percentage'])
plt.title('Percentage of Recalls by State')
plt.xlabel('State')
plt.ylabel('Percentage of Recalls')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


In [14]:
# Use Pandas to calculate the summary statistics data
summary_statistics = df.describe()

# Print the summary statistics
print(summary_statistics)


       Precipitation
count    2021.000000
mean        0.177279
std         0.461190
min         0.000000
25%         0.000000
50%         0.020000
75%         0.130000
max         6.700000


# Close Session

In [19]:
# Close Session
session.close()